# Exploratory Data Analysys

In [39]:
import pandas as pd
import numpy as np
import os
import matplotsoccer
import math

In [40]:
# constants

PATH: str = "../data/spadl_format/"
LEAGUES: list[str] = ["England", "Spain", "France", "Italy", "Germany"]

In [41]:
teams_df = pd.read_json("../data/wyscout/teams/teams.json")
teams_df

,city,name,wyId,officialName,area,type
0,Newcastle upon Tyne,Newcastle United,1613,Newcastle United FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
1,Vigo,Celta de Vigo,692,Real Club Celta de Vigo,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
2,Barcelona,Espanyol,691,Reial Club Deportiu Espanyol,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
3,Vitoria-Gasteiz,Deportivo Alav\u00e9s,696,Deportivo Alav\u00e9s,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
4,Valencia,Levante,695,Levante UD,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
...,...,...,...,...,...,...
137,Las Rozas,Spain,1598,Spain,"{'name': 'Spain', 'id': 724, 'alpha3code': 'ES...",national
138,Beograd,Serbia,17322,Serbia,"{'name': 'Serbia', 'id': 688, 'alpha3code': 'S...",national
139,Bern,Switzerland,6697,Switzerland,"{'name': 'Switzerland', 'id': 756, 'alpha3code...",national
140,Warszawa,Poland,13869,Poland,"{'name': 'Poland', 'id': 616, 'alpha3code': 'P...",national


In [42]:
df_dict = {}
for league in LEAGUES:
    df = pd.read_csv(f"{PATH}{league}.csv", index_col=0)

    # remove not used columns
    df.drop(["original_event_id", "type_name", "result_id", "bodypart_name", "bodypart_id"], inplace=True, axis=1)

    # create column with discretized positions
    # 0 is defense, 1 is midfield and 2 is attack
    df["position"] = df["start_x"].apply(lambda x: 0 if x <= 35 else 1 if x <= 70 else 2)

    # create columns with dists between start and end of play. Most important for passes right now
    def calc_dist(row):
        return math.sqrt(math.pow(row["start_x"] - row["end_x"], 2) + math.pow(row["start_y"] - row["end_y"], 2))

    df["distance"] = df.apply(calc_dist, axis=1)
    df_dict[league] = df

In [43]:
for league in LEAGUES:
    df=df_dict[league]
    vals = {}
    for team_id in df["team_id"].unique():
        aux_df = df[(df["team_id"] == team_id) & (df["type_id"] == 0) & (df["result_name"] == "success")].describe()
        vals[team_id] = aux_df["distance"].iloc[1]
    vals = {k: v for k, v in sorted(vals.items(), key=lambda item: item[1])}
    print(league)
    for k, v in vals.items():
        team_name = teams_df[teams_df["wyId"] == k]["officialName"].iloc[0]
        print(team_name + ": " + str(v))
    

England
Manchester City FC: 17.208294705045727
Arsenal FC: 17.729418216599584
Manchester United FC: 17.82323168977544
AFC Bournemouth: 18.416690138870038
Tottenham Hotspur FC: 18.49016291527559
Crystal Palace FC: 18.55517231543003
Chelsea FC: 18.682360085097265
Southampton FC: 18.80086498340964
Liverpool FC: 18.808336452956677
West Ham United FC: 18.832440595084144
Watford FC: 18.966950078343636
Newcastle United FC: 18.972262064832034
Swansea City AFC: 19.077093616506826
Leicester City FC: 19.188863825446003
Huddersfield Town FC: 19.214874210487316
Stoke City FC: 19.690287465628373
West Bromwich Albion FC: 19.712070043795737
Everton FC: 19.785332507800135
Brighton & Hove Albion FC: 19.83836518037719
Burnley FC: 20.6337458098742
Spain
Club Atl\u00e9tico de Madrid: 17.58148956397191
FC Barcelona: 17.618442299406397
UD Las Palmas: 17.864495605281746
Real Madrid Club de F\u00fatbol: 17.904624229104698
Sevilla FC: 18.06959657763688
Real Betis Balompi\u00e9: 18.65881369540685
Villarreal Club

In [44]:
vals

{2975: 18.12333922270929,
 2446: 18.319082224155302,
 2447: 18.3861534294937,
 2454: 18.648236000678743,
 2444: 18.83535246198951,
 2482: 19.136350656532308,
 2449: 19.25475249399921,
 2455: 19.26907457215873,
 2445: 19.616463757324798,
 2463: 19.625688814973593,
 2457: 19.762082506207097,
 2450: 19.83371983001362,
 2451: 19.836122251164785,
 2443: 19.8828072867746,
 2460: 19.88431335865311,
 2453: 19.903781213215726,
 2481: 20.22832341838926,
 2462: 20.60579110372748}

In [45]:
df[(df["team_id"] == 1625) & (df["position"] == 2)] # fuck yes

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,type_id,action_id,result_name,player_name,position,distance


In [46]:
df = df.groupby(["game_id"])
